----   
Lesson 10: Defi & Aave    
---         

> this notebook *translated* the code from [Solidity, Blockchain, and Smart Contract Course, Lesson 10: Defi & Aave](https://www.youtube.com/watch?v=M576WGiDBdQ&t=30893s)   
> The students learn:   
>   a) how get `WETH` into their `Metamask` wallet and    
>   b) how to borrow (and repay) `ETH` on `aave`.   
----     

In [156]:
# IMPORT LIBRARIES
import os
import sys
from web3 import Web3 
from web3.gas_strategies.time_based import medium_gas_price_strategy # 👈🏻 NEW
from dotenv import load_dotenv
import pkg_resources

# UnpackAI Trainings Module
import bl101

# Constants
ENVIRONMENT = pkg_resources.resource_filename('bl101', '.env')

# Load environment variables from .env in bl101
load_dotenv(ENVIRONMENT)

# LOAD METAMASK CREDENTIALS from local file
load_dotenv('.privatecredentials')
private_key = "0x" + os.getenv("private_key_A")
account_1 = os.getenv("wallet_address_A1")
infura_url_kovan = os.getenv("infura_kovan")

In [157]:
#  ABI for getting weth and borrowing
sys.path.insert(0, './helpers')
from abis import weth_abi
from abis import lending_pool_addresses_provider_abi
from abis import lending_pool_abi
from abis import erc20_abi
from abis import price_feed_abi

# Getting `WETH`
> Minting WETH by depositing ETH

Wrapped Ethereum (WETH) is used to buy and sell with auctions on OpenSea. WETH is a currency that allows users to make pre-authorized bids that can be fulfilled at a later date without any further action from the bidder. ETH and WETH are worth exactly the same amount and can be exchanged directly on your OpenSea profile.

>*The following code explains how to get WETH to your Metamask walet in `web3.py`*

In [158]:
# We are going to borrow:
weth_I_want = 0.001 #WETH

Unfortunately, Eth for `Kovan` is a little bit more difficult to get than for `Rinkeby`. You can get `Kovan` Eths from a couple of sites:
1. [Kovan Faucet](https://faucet.kovan.network/)
    - site was offline recently 😕
2. [Gitter](https://gitter.im/kovan-testnet/faucet#)
    - you need to sign on with your `twitter`, `github` or `gitlab` account
    - then click on `join room` at the buttom of the page and paste in your wallet address
    - only 0.001 Eth and you have to wait 24h to claim ETH again
3. [Ethdrop](https://ethdrop.dev/)
    - very simply: just paste your wallet address
    - only 0.001 Eth and you have to wait 24h to claim ETH again

In [159]:
# Connect to Kovan Testnet via Infura
provider = 'https://kovan.infura.io/v3/' + os.environ.get("INFURA")  #my Infura project ID/uri for Rinkeby (stored in bl101)
w3 = Web3(Web3.HTTPProvider(provider))
w3.isConnected()

True

In [160]:
# weth_token
weth_address = Web3.toChecksumAddress("0xd0a1e359811322d97991e03f863a0c30c2cf029c")
weth = w3.eth.contract(address=weth_address, abi=weth_abi)

In [161]:
# deposit function in weth_abi
function_call = weth.functions.deposit()

In [162]:
amount = w3.toWei(weth_I_want, "ether")
nonce = w3.eth.getTransactionCount(account_1)
transaction = function_call.buildTransaction(
        {
            "chainId": 42,
            "from": account_1,
            "nonce": nonce,
            "value": amount,
        }
    )

In [164]:
 # Sign transaction
 signed_txn = w3.eth.account.sign_transaction(transaction, private_key=private_key)

In [165]:
# Send transaction
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
print(f"Transaction (tx) hash: {tx_hash.hex()}")

Transaction (tx) hash: 0x12694dd3e16b9df64e96b7c65ad35909a2e39b8c13fc951a5c4bedf8168f4f1c


In [166]:
w3.eth.wait_for_transaction_receipt(tx_hash)
print(f"Received {weth_I_want} WETH")

Received 0.001 WETH


Checking our `Metamask` wallet, we can see that we received 0.01 WETH in exchange to 0.01 ETH.
![Metamask wallets before and after transaction](images/getting_weth.png)

As you might notice, our Ether decreased by a little bit more than 0.01 Ether. Thats the transaction (gas Price * gas). We can also derrive the transaction fee from the transaction:

In [167]:
# See transaction fee
tx = w3.eth.get_transaction(tx_hash)
transaction_fee = tx["gas"] * tx["gasPrice"] / 10**18
print(f"Transaction Fee: {transaction_fee:02f} Wei")

Transaction Fee: 0.000070 Wei


We can also check our transaction on the [Koven Testnet](https://kovan.etherscan.io/) by searching for our `transaction hash`:

![Receipt for Getting Weth](images/getting_weth_txreceipt.png)

# Borrowing `LINK`

In [102]:
# Although we are probably still connected ... Let's connect again to Kovan Testnet via Infura
infura_url_rinkeby = 'https://kovan.infura.io/v3/' + os.environ.get("INFURA")  #my Infura project ID/uri for Rinkeby (stored in bl101)
w3 = Web3(Web3.HTTPProvider(infura_url_rinkeby))
w3.isConnected()

True

### Get Lending Pool

In [168]:
w3.eth.set_gas_price_strategy(medium_gas_price_strategy)

In [169]:
lending_pool_addresses_provider_address = Web3.toChecksumAddress("0x88757f2f99175387aB4C6a4b3067c77A695b0349")
lending_poll_addresses_provider = w3.eth.contract(
        address=lending_pool_addresses_provider_address,
        abi=lending_pool_addresses_provider_abi,
    )
lending_pool_address = (
        lending_poll_addresses_provider.functions.getLendingPool().call()
    )

So where is the ABI coming from? We stored it in our `bl101` package and we loaded it at the beginning into `lending_pool_addresses_provider_abi`. But here we are interested in the function `getLendingPool`:  
```
{
    "inputs": [],
    "name": "getLendingPool",
    "outputs": [{"internalType": "address", "name": "", "type": "address"}],
    "stateMutability": "view",
    "type": "function",
}
```
*We can see the function has no input parameters and 1 output value of `type` address.* 

### Approve `ERC20`

In [170]:
weth_token_address = Web3.toChecksumAddress("0xd0a1e359811322d97991e03f863a0c30c2cf029c")
nonce = w3.eth.getTransactionCount(account_1)
erc20 = w3.eth.contract(address=weth_token_address, abi=erc20_abi)

In [171]:
amount = w3.toWei(0.01, "ether")
function_call = erc20.functions.approve(lending_pool.address, amount)

In [172]:
# Get the new nonce
nonce = w3.eth.getTransactionCount(account_1)
print("Nonce: ", nonce)

# Build Transaction
transaction = function_call.buildTransaction(
        {
            "chainId": 42,
            "from": account_1,
            "nonce": nonce,
        }
    )
# Sign Transaction
signed_txn = w3.eth.account.sign_transaction(transaction, private_key=private_key)
    
# Send Transaction
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
w3.eth.wait_for_transaction_receipt(tx_hash)

print(f"Approved {amount} tokens for contract {lending_pool.address}")

Nonce:  34
Approved 10000000000000000 tokens for contract 0xE0fBa4Fc209b4948668006B2bE61711b7f465bAe


### Depositing to `AAVE`

In [173]:
# deposit function of lenging_pool contract
function_call = lending_pool.functions.deposit(
        weth_address, amount, account_1, 0)

In [174]:
# Get the new nonce
nonce = w3.eth.getTransactionCount(account_1)
print("Nonce: ", nonce)

# Build Transaction
transaction = function_call.buildTransaction(
        {
            "chainId": 42,
            "from": account_1,
            "nonce": nonce,
        }
    )

# Sign Transaction
signed_txn = w3.eth.account.sign_transaction(transaction, private_key=private_key)

# Send Transaction
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
w3.eth.wait_for_transaction_receipt(tx_hash)

Nonce:  35


AttributeDict({'blockHash': HexBytes('0x6fb6794dd68c76d3a1c6b51c9209afe2ea9d923c7fef0c5271757a070eb2ab5b'),
 'blockNumber': 31648376,
 'contractAddress': None,
 'cumulativeGasUsed': 369940,
 'effectiveGasPrice': 1500000007,
 'from': '0x7Dda3ae0b40D7d86AC54edc7888c40EF7521c3c0',
 'gasUsed': 181704,
 'logs': [AttributeDict({'address': '0x87b1f4cf9BD63f7BBD3eE1aD04E8F52540349347',
   'blockHash': HexBytes('0x6fb6794dd68c76d3a1c6b51c9209afe2ea9d923c7fef0c5271757a070eb2ab5b'),
   'blockNumber': 31648376,
   'data': '0x000000000000000000000000000000000000000000000000017416815ce8d8d7',
   'logIndex': 2,
   'removed': False,
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
    HexBytes('0x000000000000000000000000464c71f6c2f760dda6093dcb91c24c39e5d6e18c')],
   'transactionHash': HexBytes('0x199cad6d2051b9d58bf6d44236ea92d46e5c7cb83d6a7f01a45a1898160cda41'),
   'transa

In [175]:
print(f"The deposit transaction hex: {tx_hash.hex()}")

The deposit transaction hex: 0x199cad6d2051b9d58bf6d44236ea92d46e5c7cb83d6a7f01a45a1898160cda41


![deposit transaction receipt](images/deposit_txreceipt.png)

### (Finally) borrowing

In [176]:
# getUserAccountData function from lending_pool contract
(
        total_collateral_eth,
        total_debt_eth,
        available_borrow_eth,
        current_liquidation_threshold,
        tlv,
        health_factor,
    ) = lending_pool.functions.getUserAccountData(account_1).call()

In [177]:
available_borrow_eth = float(Web3.fromWei(available_borrow_eth, "ether"))
total_collateral_eth = float(Web3.fromWei(total_collateral_eth, "ether"))
total_debt_eth = float(Web3.fromWei(total_debt_eth, "ether"))
print(f"You have {total_collateral_eth} worth of ETH deposited.")
print(f"You have {total_debt_eth} worth of ETH borrowed.")
print(f"You can borrow {available_borrow_eth} worth of ETH.")

You have 0.3785671307172159 worth of ETH deposited.
You have 1.87363573865e-07 worth of ETH borrowed.
You can borrow 0.3028535172101989 worth of ETH.


*So, now know hoch much we can borrow.*

#### Get Asset Price

In [178]:
link_eth_price_feed = w3.eth.contract(
        address="0x3Af8C569ab77af5230596Acf0E8c2F9351d24C38", #link_eth_price contract
        abi=price_feed_abi)

# function latestRoundData from lenk_eth_price contract        
latest_price = float(Web3.fromWei(
        link_eth_price_feed.functions.latestRoundData().call()[1], "ether"
    ))
print(f"The LINK/ETH price is {latest_price}")

The LINK/ETH price is 0.003738343147504658


In [179]:
# In case we want to borrow all (95%)
amount_erc20_to_borrow = (1 / latest_price) * (available_borrow_eth * 0.95)
print(f"We are going to borrow {amount_erc20_to_borrow} LINK")


We are going to borrow 76.96212733754412 LINK


In [180]:
# ERC20 address
aave_link_token_address = "0xAD5ce863aE3E4E9394Ab43d4ba0D80f419F61789"



In [181]:
# borrow function from lending_pool contract
function_call = lending_pool.functions.borrow(
        aave_link_token_address, Web3.toWei(amount_erc20_to_borrow, "ether"), 1, 0, account_1
    ) # 1 is stable interest rate and 0 is the referral code


In [182]:
# Get the new nonce
nonce = w3.eth.getTransactionCount(account_1)
print("Nonce: ", nonce)

# Build Transaction
transaction = function_call.buildTransaction(
        {
            "chainId": 42,
            "from": account_1,
            "nonce": nonce,
        }
    )

# Sign Transaction
signed_txn = w3.eth.account.sign_transaction(transaction, private_key=private_key)

# Send Transaction
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
w3.eth.wait_for_transaction_receipt(tx_hash)
print(f"Congratulations! We have just borrowed {amount_erc20_to_borrow} Link")

Nonce:  36
Congratulations! We have just borrowed 76.96212733754412


NameError: name 'borrow_tx' is not defined

In [183]:
print(f"Here is the borrow transaction hex: {tx_hash.hex()}")

Here is the borrow transaction hex: 0x04ed1f089bf654d9e242fb4646c041e87258154103dcd76518ee502bb9c06cd9


![borrow transaction receipt](images/borrow_txreceipt.png)

## Repay all

In [196]:
# How much do we have to repay in ETHER?
amount_to_repay = (1 / latest_price) * (total_debt_eth * 0.95)
print(f"Amount to repay: {amount_to_repay:.5f} ETH (or {w3.toWei(amount_to_repay, 'ether')} WEI)") 

Amount to repay: 0.00005 ETH (or 47613444819949 WEI)


> first, we have again approve `ERC20` (see above)

In [193]:
# Get the new nonce
nonce = w3.eth.getTransactionCount(account_1)
print("Nonce: ", nonce)

erc20 = w3.eth.contract(address=erc20_address, abi=erc20_abi)
function_call = erc20.functions.approve(spender, amount)
nonce = w3.eth.getTransactionCount(my_address)
transaction = function_call.buildTransaction(
    {
        "chainId": config["networks"][network]["chain_id"],
        "from": my_address,
        "nonce": nonce,
    }
)
signed_txn = w3.eth.account.sign_transaction(
    transaction, private_key=os.getenv("PRIVATE_KEY")
)
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
w3.eth.wait_for_transaction_receipt(tx_hash)
print(f"Approved {amount} tokens for contract {spender}")


1.028044365563781

### Approve `ERC20`

In [198]:
erc20 = w3.eth.contract(address=aave_link_token_address, abi=erc20_abi)

In [207]:
amount_to_repay

4.761344481994967e-05

In [199]:
function_call = erc20.functions.approve(lending_pool.address,  w3.toWei(amount_to_repay, 'ether'))

In [200]:
# Get the new nonce
nonce = w3.eth.getTransactionCount(account_1)
print("Nonce: ", nonce)

# Build Transaction
transaction = function_call.buildTransaction(
        {
            "chainId": 42,
            "from": account_1,
            "nonce": nonce,
        }
    )
# Sign Transaction
signed_txn = w3.eth.account.sign_transaction(transaction, private_key=private_key)
    
# Send Transaction
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
w3.eth.wait_for_transaction_receipt(tx_hash)

print(f"Approved {amount} tokens for contract {lending_pool.address}")

Nonce:  37
Approved 10000000000000000 tokens for contract 0xE0fBa4Fc209b4948668006B2bE61711b7f465bAe


... and finally repaying by calling the `repay` function from `lending_pool` contract:

In [203]:
function_call = lending_pool.functions.repay(
        aave_link_token_address,
        Web3.toWei(amount_to_repay, "ether"),
        # the ratemode
        1,
        account_1,
    )

In [205]:
# Get the new nonce
nonce = w3.eth.getTransactionCount(account_1)
print("Nonce: ", nonce)

transaction = function_call.buildTransaction(
        {
            "chainId": 42,
            "from": account_1,
            "nonce": nonce,
        }
    )
print("Repaying...")
signed_txn = w3.eth.account.sign_transaction(
    transaction, private_key=private_key)


Nonce:  38
Repaying...


In [206]:
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
w3.eth.wait_for_transaction_receipt(tx_hash)
print("Repaid!")
print(f"Here is the repay transaction hex: {tx_hash.hex()}")

Repaid!
Here is the repay transaction hex: 0x5159830b94a7ba4e48e13fd95026c0394938fc0637a8cea917772ded6f389c5e


![Repay receipt](images/repaying_txreceipt.png)

# 📚 Reference

<a name="cite_note-1"></a>1. [^](#cite_ref-1) Github repository [🌏 aave_web3_py](https://github.com/PatrickAlphaC/aave_web3_py)